The <b>Pattern Matcher</b> in OpenCog is commonly used as a tool to search the AtomSpace for specific patterns or "templates" of Atoms.

It can also be considered as a form of fill-in-the-blanks operation.

Given a template pattern, with "blanks" in it, a search is performed for all graphs that match, and can "fill in" those "blanks".

Each "blank" is called a "variable", and each (sub-)graph that can "fit" into that "blank" is called a "grounding".

The pattern matcher finds groundings for variables.

In [6]:
import common
from opencog.atomspace import AtomSpace, types
from opencog.bindlink import execute_atom
from opencog.type_constructors import *
from opencog.utilities import initialize_opencog

atomspace = AtomSpace()
initialize_opencog(atomspace)

Let's populate the AtomSpace with some knowledge:

In [9]:
# Animals
InheritanceLink(ConceptNode("cat"), ConceptNode("animal"))
InheritanceLink(ConceptNode("dog"), ConceptNode("animal"))
InheritanceLink(ConceptNode("rat"), ConceptNode("animal"))
InheritanceLink(ConceptNode("fish"), ConceptNode("animal"))

# Colors
InheritanceLink(ConceptNode("red"), ConceptNode("color"))
InheritanceLink(ConceptNode("green"), ConceptNode("color"))
InheritanceLink(ConceptNode("blue"), ConceptNode("color"))
InheritanceLink(ConceptNode("orange"), ConceptNode("color"))

# Fruits
InheritanceLink(ConceptNode("apple"), ConceptNode("fruit"))
InheritanceLink(ConceptNode("orange"), ConceptNode("fruit"))
InheritanceLink(ConceptNode("kiwi"), ConceptNode("fruit"))
InheritanceLink(ConceptNode("banana"), ConceptNode("fruit"))

(InheritanceLink
  (ConceptNode "banana") ; [574648249734392479][1]
  (ConceptNode "fruit") ; [4059050663830746403][1]
) ; [16595301519335773200][1]

<b>BindLink</b> provides an easy-to-use, convenient API to the pattern matcher in OpenCog, and it has the following structure:

```
BindLink
  variable-declarations (optional)
  pattern-to-be-matched
  instantiated-pattern
```

Now, let's say we want to find all of the animals in the AtomSpace, how can we use <b>BindLink</b> for that?

Let's start with the variable first:

In [17]:
var_x = VariableNode("$x")
var_decl = TypedVariableLink(var_x, TypeNode("ConceptNode"))
print(var_decl)

(TypedVariableLink
  (VariableNode "$x")
  (TypeNode "ConceptNode")
)



Then the pattern:

In [18]:
pattern = InheritanceLink(var_x, ConceptNode("animal"))
print(pattern)

(InheritanceLink
  (VariableNode "$x")
  (ConceptNode "animal")
)



Finally, putting things all together:

In [16]:
bindlink = BindLink(var_decl, pattern, var_x)
print(bindlink)

(BindLink
  (TypedVariableLink
    (VariableNode "$x")
    (TypeNode "ConceptNode")
  )
  (InheritanceLink
    (VariableNode "$x")
    (ConceptNode "animal")
  )
  (VariableNode "$x")
)



In [23]:
result = execute_atom(atomspace, bindlink)
print(result)

(SetLink
  (ConceptNode "rat")
  (ConceptNode "cat")
  (ConceptNode "dog")
  (ConceptNode "fish")
)



Moving forward, let's say we want to see if there is any concept that inherits from both <b>color</b> and <b>fruit</b>, how do we do that?

For the variable, we can just reuse `var_x`, no changes is needed for that.

For the pattern, let's define each of the clauses:

In [24]:
clause_1 = InheritanceLink(var_x, ConceptNode("color"))
clause_2 = InheritanceLink(var_x, ConceptNode("fruit"))

Since we now have two clauses, an <b>AndLink</b> is needed to wrap them up:

In [26]:
pattern = AndLink(clause_1, clause_2)
print(pattern)

(AndLink
  (InheritanceLink
    (VariableNode "$x")
    (ConceptNode "fruit")
  )
  (InheritanceLink
    (VariableNode "$x")
    (ConceptNode "color")
  )
)



Put them into a <b>BindLink</b>:

In [28]:
bindlink = BindLink(var_decl, pattern, var_x)
print(bindlink)

(BindLink
  (TypedVariableLink
    (VariableNode "$x")
    (TypeNode "ConceptNode")
  )
  (AndLink
    (InheritanceLink
      (VariableNode "$x")
      (ConceptNode "fruit")
    )
    (InheritanceLink
      (VariableNode "$x")
      (ConceptNode "color")
    )
  )
  (VariableNode "$x")
)



And search for the result:

In [29]:
result = execute_atom(atomspace, bindlink)
print(result)

(SetLink
  (ConceptNode "orange")
)

